In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import LinearNDInterpolator
import os
from tqdm import tqdm
import datetime



def interpolate_era_5(era_5_df, cygnss_row, component):
    
    if component == 'v10':
        interp_v10 = LinearNDInterpolator(list(zip(era_5_df['sp_lon'], era_5_df['sp_lat'], era_5_df['hours_since_ref'])),
                                          era_5_df['v10'])
        return interp_v10(cygnss_row[2], cygnss_row[1], cygnss_row[3])

    
    else:

        interp_u10 = LinearNDInterpolator(list(zip(era_5_df['sp_lon'], era_5_df['sp_lat'], era_5_df['hours_since_ref'])),
                                          era_5_df['u10'])
        return interp_u10(cygnss_row[2], cygnss_row[1], cygnss_row[3])
    
    

def interpolate_oskar(oskar_df, cygnss_row, component):
    
    if component == 'v':
        interp_v = LinearNDInterpolator(list(zip(oskar_df['lat'], oskar_df['lon'], oskar_df['time'])),oskar_df['v'])

        return interp_v(cygnss_row[2], cygnss_row[1], cygnss_row[3])

    
    else:

        interp_u = LinearNDInterpolator(list(zip(oskar_df['lat'], oskar_df['lon'], oskar_df['time'])),oskar_df['u'])

        return interp_u(cygnss_row[2], cygnss_row[1], cygnss_row[3])


    

def get_interp_cygnnss(cygnss_row, era_5_df, oskar_df):

    v10 = interpolate_era_5(era_5_df, cygnss_row, 'v10')
    u10 = interpolate_era_5(era_5_df, cygnss_row, 'u10')
    v_current = interpolate_oskar(oskar_df, cygnss_row, 'v')
    u_current = interpolate_oskar(oskar_df, cygnss_row, 'u')
        
    diff_u = u10 - u_current
    diff_v = v10 - v_current
    
    total_wind = np.sqrt(u10 ** 2 + v10 ** 2)
    delta = np.sqrt(diff_u ** 2 + diff_v ** 2)

    return total_wind, delta


def get_relevant_filenames(cygnss_filename, data_set):
    relevant_filenames = [cygnss_filename ]
    date = datetime.datetime(int(cygnss_filename[0:4]), int(cygnss_filename[5:7]), int(cygnss_filename[8:10]))
    if data_set == "era_5":
        next_date = date + datetime.timedelta(days=1)
        next_day_string = str(next_date.year) + "_" + str(next_date.month).zfill(2) + '_' + str(next_date.day).zfill(2) +".csv"
        relevant_filenames.append(next_day_string)
        return relevant_filenames
    else:
        for i in range(-7, 0):
            next_date = date + datetime.timedelta(days=i)
            next_day_string = str(next_date.year) + "_" + str(next_date.month).zfill(2) + '_' + str(next_date.day).zfill(2)+".csv"
            relevant_filenames.append(next_day_string)
        date = datetime.datetime(int(cygnss_filename[0:4]), int(cygnss_filename[5:7]), int(cygnss_filename[8:10]))
        for i in range(1, 8):
            next_date = date + datetime.timedelta(days=i)
            next_day_string = str(next_date.year) + "_" + str(next_date.month).zfill(2) + '_' + str(next_date.day).zfill(2)+".csv"
            relevant_filenames.append(next_day_string)
        return relevant_filenames


def relevant_files(input_arguments, directory):
    files = os.listdir(directory)    
    # Sort file names by name
    files = sorted(files) 

    relevant_files = []
    for index, file_name in enumerate(files):
        for argument in input_arguments:
            if file_name.startswith(argument):
                relevant_files.append(file_name)   
    relevant_files = [directory + '/' + s for s in relevant_files]

    return relevant_files

In [2]:
files = os.listdir("level_2_mss")
for file in tqdm(files):
    total_winds = []
    deltas = []
    potensial_era_5_files = get_relevant_filenames(file, 'era_5')
    potensial_oskar_files = get_relevant_filenames(file, 'oskar')
        
    era_5_files = relevant_files(potensial_era_5_files, 'era_5')
    oskar_files = relevant_files(potensial_oskar_files, 'oskar_data')

    print("Processing these files : ")
    print(file)
    print(era_5_files)
    print(oskar_files)
    
    cygnss_df = pd.read_csv("level_2_mss/" + file)
    era_5_df =  pd.concat(map(pd.read_csv, era_5_files ))
    oskar_df =  pd.concat(map(pd.read_csv, oskar_files ))
    if len(era_5_files) > 1 and len(oskar_files) > 1:
        
        for index, row in tqdm(cygnss_df.iterrows()):
            possible_lats_oskar = [row[1] - 0.35, row[1] + 0.35] 
            possible_lons_oskar = [row[2] - 0.35, row[2] + 0.35] 

            possible_lats_era5 = [row[1] - 0.25, row[1] + 0.25] 
            possible_lons_era5 = [row[2] - 0.25, row[2] + 0.25] 
            possible_times_era5 =[row[3] - 2, row[3] + 2]

            sub_era_5_df = era_5_df.loc[(era_5_df['hours_since_ref']>possible_times_era5[0]) & (era_5_df['hours_since_ref']< possible_times_era5[1])
                                        & (era_5_df['sp_lon']>possible_lons_era5[0]) & (era_5_df['sp_lon']< possible_lons_era5[1]) &
                                        (era_5_df['sp_lat']>possible_lats_era5[0]) & (era_5_df['sp_lat']< possible_lats_era5[1])]

            
            
            sub_oskar_df = oskar_df.loc[(oskar_df['lon']>possible_lons_oskar[0]) & (oskar_df['lon']< possible_lons_oskar[1]) &
                                                 (oskar_df['lat']>possible_lats_oskar[0]) & (oskar_df['lat']< possible_lats_oskar[1])]
                        
            total_wind, delta = get_interp_cygnnss(row,sub_era_5_df, sub_oskar_df)
        cygnss_df['total_wind'] = total_winds
        cygnss_df['delta'] = deltas
        cygnss_df.to_csv("colocated_data/" + file[0:-4] + ".csv" ,index=False)

  0%|          | 0/143 [00:00<?, ?it/s]

Processing these files : 
2017_03_18.csv
['era_5/2017_03_18.csv', 'era_5/2017_03_19.csv']
['oskar_data/2017_03_18.csv', 'oskar_data/2017_03_23.csv']



0it [00:00, ?it/s]
1it [00:00,  3.97it/s]

Interp v10
901896    210.75
901897    210.75
901920    210.75
901921    210.75
909216    211.00
909217    211.00
909240    211.00
909241    211.00
Name: sp_lon, dtype: float64 901896    22.00
901897    22.00
901920    21.75
901921    21.75
909216    22.00
909217    22.00
909240    21.75
909241    21.75
Name: sp_lat, dtype: float64 901896    214320.0
901897    214321.0
901920    214320.0
901921    214321.0
909216    214320.0
909217    214321.0
909240    214320.0
909241    214321.0
Name: hours_since_ref, dtype: float64
Interp u10
Interp v
Interp u



2it [00:00,  3.99it/s]

...

Interp v10
6644184    46.75
6644185    46.75
6644186    46.75
6644208    46.75
6644209    46.75
6644210    46.75
6651504    47.00
6651505    47.00
6651506    47.00
6651528    47.00
6651529    47.00
6651530    47.00
Name: sp_lon, dtype: float64 6644184   -13.50
6644185   -13.50
6644186   -13.50
6644208   -13.75
6644209   -13.75
6644210   -13.75
6651504   -13.50
6651505   -13.50
6651506   -13.50
6651528   -13.75
6651529   -13.75
6651530   -13.75
Name: sp_lat, dtype: float64 6644184    214320.0
6644185    214321.0
6644186    214322.0
6644208    214320.0
6644209    214321.0
6644210    214322.0
6651504    214320.0
6651505    214321.0
6651506    214322.0
6651528    214320.0
6651529    214321.0
6651530    214322.0
Name: hours_since_ref, dtype: float64
Interp u10
Interp v
Interp u



18it [00:04,  4.35it/s]

Interp v10
901896    210.75
901897    210.75
901898    210.75
901920    210.75
901921    210.75
901922    210.75
909216    211.00
909217    211.00
909218    211.00
909240    211.00
909241    211.00
909242    211.00
Name: sp_lon, dtype: float64 901896    22.00
901897    22.00
901898    22.00
901920    21.75
901921    21.75
901922    21.75
909216    22.00
909217    22.00
909218    22.00
909240    21.75
909241    21.75
909242    21.75
Name: sp_lat, dtype: float64 901896    214320.0
901897    214321.0
901898    214322.0
901920    214320.0
901921    214321.0
901922    214322.0
909216    214320.0
909217    214321.0
909218    214322.0
909240    214320.0
909241    214321.0
909242    214322.0
Name: hours_since_ref, dtype: float64
Interp u10
Interp v
Interp u


18it [00:04,  3.94it/s]
  0%|          | 0/143 [00:15<?, ?it/s]


KeyboardInterrupt: 